# Environmental Sensor Data Analysis with AWS

This notebook demonstrates a complete environmental monitoring workflow using AWS services:
- Generate synthetic sensor data (air quality, water quality, weather)
- Upload data to S3
- Process with Lambda (AQI/WQI calculation, alerts)
- Query results from DynamoDB
- Visualize trends and patterns

**Prerequisites:**
- AWS account configured
- Completed setup_guide.md
- All AWS resources created (S3, Lambda, DynamoDB, SNS)

**Duration:** 30-45 minutes

## 1. Setup and Configuration

In [ ]:
# Import libraries
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import json
import sys
from pathlib import Path

# Add scripts directory to path
sys.path.append(str(Path('..') / 'scripts'))

# Configure matplotlib
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

print("✓ Libraries imported successfully")

In [ ]:
# AWS Configuration
# TODO: Update these values with your AWS resources

BUCKET_NAME = 'environmental-data-YOUR-BUCKET'  # Your S3 bucket
DYNAMODB_TABLE = 'EnvironmentalReadings'        # DynamoDB table name
LAMBDA_FUNCTION = 'process-sensor-data'          # Lambda function name
REGION = 'us-east-1'                             # AWS region

# Initialize AWS clients
s3_client = boto3.client('s3', region_name=REGION)
dynamodb = boto3.resource('dynamodb', region_name=REGION)
lambda_client = boto3.client('lambda', region_name=REGION)

print(f"✓ AWS clients initialized")
print(f"  Region: {REGION}")
print(f"  Bucket: {BUCKET_NAME}")
print(f"  DynamoDB: {DYNAMODB_TABLE}")
print(f"  Lambda: {LAMBDA_FUNCTION}")

## 2. Generate Sample Environmental Data

Generate synthetic sensor data with realistic patterns:
- Diurnal variations (day/night cycles)
- Rush hour effects (air quality)
- Photosynthesis effects (water quality)
- Random anomalies and pollution events

In [ ]:
from upload_to_s3 import EnvironmentalDataGenerator

# Initialize data generator
generator = EnvironmentalDataGenerator(seed=42)

# Generate data for 7 days
DAYS = 7

print(f"Generating {DAYS} days of sensor data...")
air_data = generator.generate_air_quality_data(days=DAYS)
water_data = generator.generate_water_quality_data(days=DAYS)
weather_data = generator.generate_weather_data(days=DAYS)

print(f"\n✓ Data generation complete:")
print(f"  Air quality: {len(air_data)} readings from {len(generator.locations)} stations")
print(f"  Water quality: {len(water_data)} readings from {len(generator.water_locations)} sites")
print(f"  Weather: {len(weather_data)} readings")

In [ ]:
# Preview air quality data
print("Air Quality Data Sample:")
air_data.head()

In [ ]:
# Preview water quality data
print("Water Quality Data Sample:")
water_data.head()

## 3. Upload Data to S3

Upload generated data to S3. This will automatically trigger Lambda processing.

In [ ]:
from upload_to_s3 import S3Uploader

# Initialize uploader
uploader = S3Uploader(BUCKET_NAME, region=REGION)

# Upload air quality data
timestamp = datetime.utcnow().strftime('%Y%m%d_%H%M%S')
s3_key = f'raw/air_quality_{timestamp}.csv'
print(f"Uploading air quality data to s3://{BUCKET_NAME}/{s3_key}")
uploader.upload_dataframe(air_data, s3_key, format='csv')

# Upload water quality data
s3_key = f'raw/water_quality_{timestamp}.csv'
print(f"Uploading water quality data to s3://{BUCKET_NAME}/{s3_key}")
uploader.upload_dataframe(water_data, s3_key, format='csv')

# Upload weather data
s3_key = f'raw/weather_{timestamp}.csv'
print(f"Uploading weather data to s3://{BUCKET_NAME}/{s3_key}")
uploader.upload_dataframe(weather_data, s3_key, format='csv')

print("\n✓ All data uploaded to S3")
print("\n⏳ Lambda processing triggered automatically...")
print("   Wait 30-60 seconds for processing to complete")

In [ ]:
# List uploaded files
print("\nUploaded files in S3:")
files = uploader.list_uploaded_files(prefix='raw/')

## 4. Monitor Lambda Processing

Check Lambda execution logs and status.

In [ ]:
import time

# Wait for Lambda processing
print("Waiting for Lambda processing (30 seconds)...")
time.sleep(30)

# Check Lambda function status
try:
    response = lambda_client.get_function(FunctionName=LAMBDA_FUNCTION)
    print(f"\n✓ Lambda function status: {response['Configuration']['State']}")
    print(f"  Last modified: {response['Configuration']['LastModified']}")
    print(f"  Memory: {response['Configuration']['MemorySize']} MB")
    print(f"  Timeout: {response['Configuration']['Timeout']} seconds")
except Exception as e:
    print(f"⚠ Could not check Lambda status: {e}")

In [ ]:
# View recent Lambda invocations
logs_client = boto3.client('logs', region_name=REGION)
log_group = f'/aws/lambda/{LAMBDA_FUNCTION}'

try:
    # Get recent log streams
    response = logs_client.describe_log_streams(
        logGroupName=log_group,
        orderBy='LastEventTime',
        descending=True,
        limit=3
    )
    
    print(f"\nRecent Lambda invocations (last 3):")
    for stream in response['logStreams']:
        last_event = datetime.fromtimestamp(stream['lastEventTimestamp']/1000)
        print(f"  {stream['logStreamName']}: {last_event}")
        
except Exception as e:
    print(f"⚠ Could not retrieve logs: {e}")

## 5. Query Results from DynamoDB

Retrieve processed sensor readings from DynamoDB.

In [ ]:
from query_results import EnvironmentalDataQuery, convert_decimal

# Initialize query client
query_client = EnvironmentalDataQuery(
    table_name=DYNAMODB_TABLE,
    region=REGION
)

print("✓ DynamoDB query client initialized")

In [ ]:
# Get all unique locations
locations = query_client.get_all_locations()
print(f"\nAvailable sensor locations ({len(locations)}):")
for loc in locations:
    print(f"  - {loc}")

In [ ]:
# Query air quality station
if locations:
    # Get first air quality station
    air_stations = [loc for loc in locations if loc.startswith('station-')]
    if air_stations:
        location = air_stations[0]
        print(f"Querying location: {location}")
        
        results = query_client.query_by_location(location, days=DAYS, limit=100)
        
        print(f"\n✓ Retrieved {len(results)} readings from {location}")
        
        # Convert to DataFrame
        results_df = pd.DataFrame(convert_decimal(results))
        print(f"\nData shape: {results_df.shape}")
        results_df.head()
    else:
        print("⚠ No air quality stations found. Wait longer for processing.")
else:
    print("⚠ No data in DynamoDB yet. Wait for Lambda processing to complete.")

In [ ]:
# Query alerts
critical_alerts = query_client.query_by_alert_status('critical', limit=50)
warning_alerts = query_client.query_by_alert_status('warning', limit=50)

print(f"\nAlert Summary:")
print(f"  Critical alerts: {len(critical_alerts)}")
print(f"  Warning alerts: {len(warning_alerts)}")

if critical_alerts:
    print(f"\nCritical alerts:")
    for alert in critical_alerts[:5]:
        print(f"  {alert['timestamp']}: {alert.get('alert_message', 'No message')}")

## 6. Data Analysis and Statistics

Calculate statistics and identify trends in environmental data.

In [ ]:
# Get statistics for all locations
all_stats = {}

for location in locations:
    stats = query_client.get_statistics(location, days=DAYS)
    if stats:
        all_stats[location] = stats

print(f"\n✓ Calculated statistics for {len(all_stats)} locations")

In [ ]:
# Display statistics for air quality stations
air_stations = {k: v for k, v in all_stats.items() if k.startswith('station-')}

if air_stations:
    print("\nAir Quality Statistics:")
    print("=" * 80)
    
    summary_data = []
    for location, stats in air_stations.items():
        if 'air_quality' in stats:
            aq = stats['air_quality']
            summary_data.append({
                'Location': location,
                'Readings': stats['total_readings'],
                'Avg AQI': f"{aq.get('avg_aqi', 0):.1f}",
                'Max AQI': f"{aq.get('max_aqi', 0):.0f}",
                'Avg PM2.5': f"{aq.get('avg_pm25', 0):.2f}",
                'Alerts': stats['alerts']['critical'] + stats['alerts']['warning']
            })
    
    summary_df = pd.DataFrame(summary_data)
    display(summary_df)
else:
    print("⚠ No air quality statistics available")

In [ ]:
# Display statistics for water quality sites
water_sites = {k: v for k, v in all_stats.items() if k.startswith(('river-', 'lake-'))}

if water_sites:
    print("\nWater Quality Statistics:")
    print("=" * 80)
    
    summary_data = []
    for location, stats in water_sites.items():
        if 'water_quality' in stats:
            wq = stats['water_quality']
            summary_data.append({
                'Location': location,
                'Readings': stats['total_readings'],
                'Avg WQI': f"{wq.get('avg_wqi', 0):.1f}",
                'Max WQI': f"{wq.get('max_wqi', 0):.0f}",
                'Avg pH': f"{wq.get('avg_ph', 0):.2f}",
                'Alerts': stats['alerts']['critical'] + stats['alerts']['warning']
            })
    
    summary_df = pd.DataFrame(summary_data)
    display(summary_df)
else:
    print("⚠ No water quality statistics available")

## 7. Data Visualization

Create visualizations to identify patterns and trends.

In [ ]:
# Prepare data for visualization
if air_stations:
    # Get detailed data for first air station
    location = list(air_stations.keys())[0]
    air_readings = query_client.query_by_location(location, days=DAYS, limit=1000)
    
    # Convert to DataFrame and parse data
    air_df = pd.DataFrame(convert_decimal(air_readings))
    air_df['timestamp'] = pd.to_datetime(air_df['timestamp'])
    air_df = air_df.sort_values('timestamp')
    
    # Extract parameters
    air_df['pm25'] = air_df['parameters'].apply(lambda x: x.get('pm25', np.nan))
    air_df['pm10'] = air_df['parameters'].apply(lambda x: x.get('pm10', np.nan))
    air_df['co2'] = air_df['parameters'].apply(lambda x: x.get('co2', np.nan))
    air_df['temperature'] = air_df['parameters'].apply(lambda x: x.get('temperature', np.nan))
    air_df['humidity'] = air_df['parameters'].apply(lambda x: x.get('humidity', np.nan))
    air_df['aqi'] = air_df['calculated_metrics'].apply(lambda x: x.get('aqi', np.nan))
    
    print(f"✓ Prepared {len(air_df)} air quality readings for visualization")
else:
    print("⚠ No air quality data available for visualization")

In [ ]:
# Time series plot: PM2.5 and AQI
if air_stations and not air_df.empty:
    fig, axes = plt.subplots(2, 1, figsize=(14, 8))
    
    # PM2.5 over time
    axes[0].plot(air_df['timestamp'], air_df['pm25'], linewidth=1.5, alpha=0.8)
    axes[0].axhline(y=35.4, color='orange', linestyle='--', label='Warning threshold')
    axes[0].axhline(y=55.4, color='red', linestyle='--', label='Critical threshold')
    axes[0].set_ylabel('PM2.5 (μg/m³)', fontsize=12)
    axes[0].set_title(f'PM2.5 Levels - {location}', fontsize=14, fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # AQI over time with color zones
    axes[1].plot(air_df['timestamp'], air_df['aqi'], linewidth=1.5, alpha=0.8, color='steelblue')
    axes[1].axhspan(0, 50, alpha=0.1, color='green', label='Good')
    axes[1].axhspan(51, 100, alpha=0.1, color='yellow', label='Moderate')
    axes[1].axhspan(101, 150, alpha=0.1, color='orange', label='Unhealthy for sensitive')
    axes[1].axhspan(151, 200, alpha=0.1, color='red', label='Unhealthy')
    axes[1].set_xlabel('Time', fontsize=12)
    axes[1].set_ylabel('AQI', fontsize=12)
    axes[1].set_title('Air Quality Index (AQI)', fontsize=14, fontweight='bold')
    axes[1].legend(loc='upper right', fontsize=9)
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠ Cannot create visualization - no data available")

In [ ]:
# Diurnal pattern analysis
if air_stations and not air_df.empty:
    # Add hour of day
    air_df['hour'] = air_df['timestamp'].dt.hour
    
    # Calculate hourly averages
    hourly_avg = air_df.groupby('hour')[['pm25', 'aqi', 'co2']].mean()
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # PM2.5 by hour
    axes[0].bar(hourly_avg.index, hourly_avg['pm25'], alpha=0.7, color='steelblue')
    axes[0].set_xlabel('Hour of Day', fontsize=11)
    axes[0].set_ylabel('PM2.5 (μg/m³)', fontsize=11)
    axes[0].set_title('Average PM2.5 by Hour', fontsize=12, fontweight='bold')
    axes[0].grid(True, alpha=0.3, axis='y')
    
    # AQI by hour
    axes[1].bar(hourly_avg.index, hourly_avg['aqi'], alpha=0.7, color='orange')
    axes[1].set_xlabel('Hour of Day', fontsize=11)
    axes[1].set_ylabel('AQI', fontsize=11)
    axes[1].set_title('Average AQI by Hour', fontsize=12, fontweight='bold')
    axes[1].grid(True, alpha=0.3, axis='y')
    
    # CO2 by hour
    axes[2].bar(hourly_avg.index, hourly_avg['co2'], alpha=0.7, color='green')
    axes[2].set_xlabel('Hour of Day', fontsize=11)
    axes[2].set_ylabel('CO2 (ppm)', fontsize=11)
    axes[2].set_title('Average CO2 by Hour', fontsize=12, fontweight='bold')
    axes[2].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
    
    print("\n📊 Diurnal Pattern Insights:")
    peak_hour = hourly_avg['pm25'].idxmax()
    print(f"  - Peak PM2.5 at hour {peak_hour}:00 ({hourly_avg['pm25'].loc[peak_hour]:.1f} μg/m³)")
    print(f"  - Lowest PM2.5 at hour {hourly_avg['pm25'].idxmin()}:00 ({hourly_avg['pm25'].min():.1f} μg/m³)")
else:
    print("⚠ Cannot create diurnal pattern - no data available")

In [ ]:
# Correlation heatmap
if air_stations and not air_df.empty:
    # Select numeric columns
    numeric_cols = ['pm25', 'pm10', 'co2', 'temperature', 'humidity', 'aqi']
    corr_data = air_df[numeric_cols].dropna()
    
    if not corr_data.empty:
        correlation = corr_data.corr()
        
        plt.figure(figsize=(10, 8))
        sns.heatmap(correlation, annot=True, fmt='.2f', cmap='coolwarm', 
                    center=0, square=True, linewidths=1)
        plt.title('Environmental Parameter Correlations', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()
        
        print("\n📊 Correlation Insights:")
        print(f"  - PM2.5 vs Temperature: {correlation.loc['pm25', 'temperature']:.2f}")
        print(f"  - PM2.5 vs Humidity: {correlation.loc['pm25', 'humidity']:.2f}")
        print(f"  - AQI vs PM2.5: {correlation.loc['aqi', 'pm25']:.2f}")
    else:
        print("⚠ Not enough data for correlation analysis")
else:
    print("⚠ Cannot create correlation heatmap - no data available")

In [ ]:
# Water quality visualization (if available)
if water_sites:
    location = list(water_sites.keys())[0]
    water_readings = query_client.query_by_location(location, days=DAYS, limit=1000)
    
    if water_readings:
        # Convert to DataFrame
        water_df = pd.DataFrame(convert_decimal(water_readings))
        water_df['timestamp'] = pd.to_datetime(water_df['timestamp'])
        water_df = water_df.sort_values('timestamp')
        
        # Extract parameters
        water_df['ph'] = water_df['parameters'].apply(lambda x: x.get('ph', np.nan))
        water_df['dissolved_oxygen'] = water_df['parameters'].apply(lambda x: x.get('dissolved_oxygen', np.nan))
        water_df['turbidity'] = water_df['parameters'].apply(lambda x: x.get('turbidity', np.nan))
        water_df['wqi'] = water_df['calculated_metrics'].apply(lambda x: x.get('wqi', np.nan))
        
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        
        # pH over time
        axes[0, 0].plot(water_df['timestamp'], water_df['ph'], linewidth=1.5, color='blue')
        axes[0, 0].axhspan(6.5, 8.5, alpha=0.2, color='green', label='Acceptable range')
        axes[0, 0].set_ylabel('pH', fontsize=11)
        axes[0, 0].set_title('pH Levels', fontsize=12, fontweight='bold')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
        
        # Dissolved oxygen
        axes[0, 1].plot(water_df['timestamp'], water_df['dissolved_oxygen'], linewidth=1.5, color='cyan')
        axes[0, 1].axhline(y=5.0, color='red', linestyle='--', label='Critical threshold')
        axes[0, 1].set_ylabel('Dissolved Oxygen (mg/L)', fontsize=11)
        axes[0, 1].set_title('Dissolved Oxygen', fontsize=12, fontweight='bold')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)
        
        # Turbidity
        axes[1, 0].plot(water_df['timestamp'], water_df['turbidity'], linewidth=1.5, color='brown')
        axes[1, 0].set_xlabel('Time', fontsize=11)
        axes[1, 0].set_ylabel('Turbidity (NTU)', fontsize=11)
        axes[1, 0].set_title('Turbidity', fontsize=12, fontweight='bold')
        axes[1, 0].grid(True, alpha=0.3)
        
        # WQI
        axes[1, 1].plot(water_df['timestamp'], water_df['wqi'], linewidth=1.5, color='green')
        axes[1, 1].axhspan(0, 25, alpha=0.1, color='green', label='Excellent')
        axes[1, 1].axhspan(26, 50, alpha=0.1, color='blue', label='Good')
        axes[1, 1].axhspan(51, 75, alpha=0.1, color='yellow', label='Fair')
        axes[1, 1].axhspan(76, 100, alpha=0.1, color='red', label='Poor')
        axes[1, 1].set_xlabel('Time', fontsize=11)
        axes[1, 1].set_ylabel('WQI', fontsize=11)
        axes[1, 1].set_title('Water Quality Index', fontsize=12, fontweight='bold')
        axes[1, 1].legend(loc='upper right', fontsize=9)
        axes[1, 1].grid(True, alpha=0.3)
        
        plt.suptitle(f'Water Quality Monitoring - {location}', fontsize=14, fontweight='bold', y=1.00)
        plt.tight_layout()
        plt.show()
    else:
        print("⚠ No water quality data available")
else:
    print("⚠ No water quality sites found")

## 8. Alert Analysis

Analyze pollution alerts and threshold violations.

In [ ]:
# Count alerts by location
alert_summary = []

for location in locations:
    readings = query_client.query_by_location(location, days=DAYS, limit=1000)
    if readings:
        critical = sum(1 for r in readings if r.get('alert_status') == 'critical')
        warning = sum(1 for r in readings if r.get('alert_status') == 'warning')
        none = sum(1 for r in readings if r.get('alert_status') == 'none')
        
        alert_summary.append({
            'Location': location,
            'Total': len(readings),
            'Critical': critical,
            'Warning': warning,
            'None': none,
            'Alert Rate': f"{(critical + warning) / len(readings) * 100:.1f}%"
        })

alert_df = pd.DataFrame(alert_summary)
alert_df = alert_df.sort_values('Critical', ascending=False)

print("\nAlert Summary by Location:")
display(alert_df)

In [ ]:
# Visualize alert distribution
if not alert_df.empty:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Stacked bar chart
    alert_df.set_index('Location')[['Critical', 'Warning', 'None']].plot(
        kind='bar', stacked=True, ax=axes[0], color=['red', 'orange', 'green'], alpha=0.7
    )
    axes[0].set_ylabel('Number of Readings', fontsize=11)
    axes[0].set_title('Alerts by Location', fontsize=12, fontweight='bold')
    axes[0].legend(title='Status')
    axes[0].tick_params(axis='x', rotation=45)
    
    # Pie chart of overall alerts
    total_critical = alert_df['Critical'].sum()
    total_warning = alert_df['Warning'].sum()
    total_none = alert_df['None'].sum()
    
    axes[1].pie(
        [total_critical, total_warning, total_none],
        labels=['Critical', 'Warning', 'None'],
        colors=['red', 'orange', 'green'],
        autopct='%1.1f%%',
        startangle=90,
        alpha=0.7
    )
    axes[1].set_title('Overall Alert Distribution', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
else:
    print("⚠ No alert data available")

## 9. Export Results

Save analysis results for further use or reporting.

In [ ]:
# Create output directory
output_dir = Path('..') / 'output'
output_dir.mkdir(exist_ok=True)

timestamp = datetime.utcnow().strftime('%Y%m%d_%H%M%S')

# Export statistics
stats_file = output_dir / f'statistics_{timestamp}.json'
with open(stats_file, 'w') as f:
    json.dump(convert_decimal(all_stats), f, indent=2)
print(f"✓ Statistics exported to {stats_file}")

# Export alert summary
alert_file = output_dir / f'alert_summary_{timestamp}.csv'
alert_df.to_csv(alert_file, index=False)
print(f"✓ Alert summary exported to {alert_file}")

# Export air quality data
if air_stations and not air_df.empty:
    air_file = output_dir / f'air_quality_{timestamp}.csv'
    air_df.to_csv(air_file, index=False)
    print(f"✓ Air quality data exported to {air_file}")

print(f"\n✓ All results exported to {output_dir}/")

## 10. Summary and Key Findings

In [ ]:
print("\n" + "="*80)
print("ENVIRONMENTAL MONITORING ANALYSIS SUMMARY")
print("="*80)

print(f"\nData Overview:")
print(f"  - Monitoring period: {DAYS} days")
print(f"  - Total locations: {len(locations)}")
print(f"  - Air quality stations: {len(air_stations)}")
print(f"  - Water quality sites: {len(water_sites)}")

print(f"\nAlert Summary:")
if not alert_df.empty:
    print(f"  - Total readings: {alert_df['Total'].sum()}")
    print(f"  - Critical alerts: {alert_df['Critical'].sum()}")
    print(f"  - Warning alerts: {alert_df['Warning'].sum()}")
    print(f"  - Overall alert rate: {(alert_df['Critical'].sum() + alert_df['Warning'].sum()) / alert_df['Total'].sum() * 100:.1f}%")

if air_stations:
    print(f"\nAir Quality Insights:")
    for location, stats in list(air_stations.items())[:3]:
        if 'air_quality' in stats:
            aq = stats['air_quality']
            print(f"  {location}:")
            print(f"    - Average AQI: {aq.get('avg_aqi', 0):.1f}")
            print(f"    - Maximum AQI: {aq.get('max_aqi', 0):.0f}")
            print(f"    - Average PM2.5: {aq.get('avg_pm25', 0):.2f} μg/m³")

if water_sites:
    print(f"\nWater Quality Insights:")
    for location, stats in list(water_sites.items())[:2]:
        if 'water_quality' in stats:
            wq = stats['water_quality']
            print(f"  {location}:")
            print(f"    - Average WQI: {wq.get('avg_wqi', 0):.1f}")
            print(f"    - Average pH: {wq.get('avg_ph', 0):.2f}")

print(f"\nAWS Services Used:")
print(f"  - S3: Data storage")
print(f"  - Lambda: Serverless processing (AQI/WQI calculation)")
print(f"  - DynamoDB: Time series database")
print(f"  - SNS: Alert notifications")

print(f"\n" + "="*80)
print("Analysis complete! Next steps:")
print("  1. Review exported files in output/ directory")
print("  2. Check SNS email for any pollution alerts")
print("  3. Run cleanup_guide.md to delete AWS resources")
print("  4. Try Tier 3 for production-grade CloudFormation deployment")
print("="*80)